In [1]:
from typing import Annotated, Literal, TypedDict
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

In [2]:
embeddings = OpenAIEmbeddings()

In [3]:
model_low_temp = ChatOpenAI(model = 'gpt-5-2025-08-07')
model_high_temp = ChatOpenAI(model = 'gpt-5-2025-08-07')

In [4]:
class State(TypedDict):
    # 대화 기록
    messages: Annotated[list, add_messages]
    # 입력
    user_query: str
    # 출력
    domain: Literal['records', 'insurance']
    documents: list[Document]
    answer: str

In [5]:
class Input(TypedDict):
    user_query : str

In [6]:
class Output(TypedDict):
    documents : list[Document]
    answer : str

In [7]:
sample_docs = [
    Document(page_content='''# 환자 의료 기록
- 이름: 홍길동
- 성별: 남
- 나이: 30세
## 진료 내역
- 2021년 1월 13일: 감기로 인한 발열로 병원 방문
- 2022년 3월 15일: 비염 진단으로 약 처방
- 2022년 5월 20일: 피부과 진료 및 약 처방
- 2022년 6월 10일: 발열 및 기침으로 코로나 검사 및 음성 판정
- 2022년 7월 2일: 코로나 진단 검사 및 양성 판정
    ''',
        metadata={'domain': 'records'}
    ),
    Document(
        page_content='''# 보험 FAQ
- Q: 과거에 병력이 있는데 가입가능한가요?
- A: 치료기간, 현재 상태, 후유증 여부, 연령 등에 따라 다르므로 가입 가능 여부는 가까운 대리점에서 상담 받으세요.


- Q: 보험금 청구 방법은 어떻게 되나요??
- A: 보험금 청구는 보험금 청구서 작성 후 가까운 대리점을 통해 진행 가능합니다.


- Q: 코로나 19도 보험 적용이 되나요?
- A: 코로나 19는 보험 대상에 포함되지 않습니다.


- Q: 보험금 지급이 거부되는 경우는 어떤 경우인가요?
- A: 보험금 지급 거부 사유는 보험 계약서에 명시되어 있습니다. 자세한 사항은 계약서를 참조하세요
        ''',
        metadata={'domain': 'insurance'}
    ),
]

In [8]:
medical_records_store = InMemoryVectorStore.from_documents(sample_docs, embeddings)
medical_records_retriever = medical_records_store.as_retriever()

In [9]:
insurance_faqs_store = InMemoryVectorStore.from_documents(sample_docs, embeddings)
insurance_faqs_retriever = insurance_faqs_store.as_retriever()

In [10]:
router_prompt = SystemMessage(
    '''사용자 문의를 어느 도메인으로 라우팅할지 결정하세요. 선택할 수 있는 두 가지 도메인은 다음과 같습니다.
- records: 진단, 치료, 처방과 같은 환자의 의료 기록을 포함합니다.  
- insurance: 보험 정책, 청구, 보장에 대한 자주 묻는 질문을 포함합니다.  


도메인 이름만 출력하세요.'''
)

In [11]:
builder = StateGraph(State, input=Input, output=Output)

/tmp/ipykernel_18112/1772522937.py:1: LangGraphDeprecatedSinceV05: `input` is deprecated and will be removed. Please use `input_schema` instead. Deprecated in LangGraph V0.5 to be removed in V2.0.
  builder = StateGraph(State, input=Input, output=Output)
/tmp/ipykernel_18112/1772522937.py:1: LangGraphDeprecatedSinceV05: `output` is deprecated and will be removed. Please use `output_schema` instead. Deprecated in LangGraph V0.5 to be removed in V2.0.
  builder = StateGraph(State, input=Input, output=Output)


In [12]:
def router_node(state: State) -> State:
    user_message = HumanMessage(state['user_query'])
    messages = [router_prompt, *state['messages'], user_message]
    res = model_low_temp.invoke(messages)
    return {
        'domain': res.content,
        # 대화 기록 업데이트
        'messages': [user_message, res],
    }


In [13]:
builder.add_node('router', router_node)

In [14]:
def pick_retriever(
    #retrieve 선택 분기
    state: State,
) -> Literal['retrieve_medical_records', 'retrieve_insurance_faqs']:
    if state['domain'] == 'records':
        return 'retrieve_medical_records'
    else:
        return 'retrieve_insurance_faqs'

In [15]:
def retrieve_medical_records(state: State) -> State:
    documents = medical_records_retriever.invoke(state['user_query'])
    return {
        'documents': documents,
    }

def retrieve_insurance_faqs(state: State) -> State:
    documents = insurance_faqs_retriever.invoke(state['user_query'])
    return {
        'documents': documents,
    }


In [16]:
builder.add_node('retrieve_medical_records', retrieve_medical_records)
builder.add_node('retrieve_insurance_faqs', retrieve_insurance_faqs)

In [17]:
medical_records_prompt = SystemMessage(
    '당신은 유능한 의료 챗봇입니다. 진단, 치료, 처방과 같은 환자의 의료 기록을 기반으로 질문에 답하세요.'
)


insurance_faqs_prompt = SystemMessage(
    '당신은 유능한 의료 보험 챗봇입니다. 보험 정책, 청구 및 보장에 대한 자주 묻는 질문에 답하세요.'
)

In [18]:
def generate_answer(state: State) -> State:
   if state['domain'] == 'records':
      prompt = medical_records_prompt
   else:
      prompt = insurance_faqs_prompt
      
   
   messages = [prompt, *state['messages'], HumanMessage(f"Dvouments: {state['documents']}")]
   res = model_high_temp.invoke(messages)
   return {
        'answer': res.content,
        'response' : res
    }

In [19]:
builder.add_node('generate_answer', generate_answer)
builder.add_edge(START, 'router')
builder.add_conditional_edges('router', pick_retriever)
builder.add_edge('retrieve_medical_records', 'generate_answer')
builder.add_edge('retrieve_insurance_faqs', 'generate_answer')
builder.add_edge('generate_answer', END)   

In [20]:
graph = builder.compile()

In [21]:
graph.invoke({'user_query':'코로나도 보험 적용이 가능한가요?'})

{'documents': [Document(id='43830897-e360-424d-8743-1db1bcebb76b', metadata={'domain': 'insurance'}, page_content='# 보험 FAQ\n- Q: 과거에 병력이 있는데 가입가능한가요?\n- A: 치료기간, 현재 상태, 후유증 여부, 연령 등에 따라 다르므로 가입 가능 여부는 가까운 대리점에서 상담 받으세요.\n\n\n- Q: 보험금 청구 방법은 어떻게 되나요??\n- A: 보험금 청구는 보험금 청구서 작성 후 가까운 대리점을 통해 진행 가능합니다.\n\n\n- Q: 코로나 19도 보험 적용이 되나요?\n- A: 코로나 19는 보험 대상에 포함되지 않습니다.\n\n\n- Q: 보험금 지급이 거부되는 경우는 어떤 경우인가요?\n- A: 보험금 지급 거부 사유는 보험 계약서에 명시되어 있습니다. 자세한 사항은 계약서를 참조하세요\n        '),
  Document(id='2de9e1e8-cdf5-4145-b94b-437c55261ebe', metadata={'domain': 'records'}, page_content='# 환자 의료 기록\n- 이름: 홍길동\n- 성별: 남\n- 나이: 30세\n## 진료 내역\n- 2021년 1월 13일: 감기로 인한 발열로 병원 방문\n- 2022년 3월 15일: 비염 진단으로 약 처방\n- 2022년 5월 20일: 피부과 진료 및 약 처방\n- 2022년 6월 10일: 발열 및 기침으로 코로나 검사 및 음성 판정\n- 2022년 7월 2일: 코로나 진단 검사 및 양성 판정\n    ')],
 'answer': '아니요. 당사 보험 FAQ 기준으로 코로나19는 보험 대상에 포함되지 않습니다. \n다만 가입하신 상품이나 특약에 따라 예외가 있을 수 있으니, 정확한 보장 여부는 계약서 약관을 확인하시거나 가까운 대리점/고객센터로 문의해 주세요. 필요하시면 가입 상품명이나 증권번호를 알려주시면 확인을 도와드리겠습니다.'}

In [22]:
resp = graph.invoke({'user_query':'홍길동의 진료 내용을 알려주세요'})

In [23]:
resp['answer']

'홍길동님의 기본 정보\n- 성별: 남\n- 나이: 30세\n\n진료 내역\n- 2021-01-13: 감기로 인한 발열로 병원 방문\n- 2022-03-15: 비염 진단 및 약 처방\n- 2022-05-20: 피부과 진료 및 약 처방\n- 2022-06-10: 발열·기침으로 코로나 검사 시행 — 음성\n- 2022-07-02: 코로나 진단 검사 — 양성\n\n추가로 필요한 상세 내용이나 요약 형식이 있으면 알려주세요.'

In [24]:
resp = graph.invoke({'user_query':'질문 내역 알려주세요'})

In [25]:
resp

{'documents': [Document(id='31dedc04-d357-40b5-a08e-3ea111b0dfa9', metadata={'domain': 'insurance'}, page_content='# 보험 FAQ\n- Q: 과거에 병력이 있는데 가입가능한가요?\n- A: 치료기간, 현재 상태, 후유증 여부, 연령 등에 따라 다르므로 가입 가능 여부는 가까운 대리점에서 상담 받으세요.\n\n\n- Q: 보험금 청구 방법은 어떻게 되나요??\n- A: 보험금 청구는 보험금 청구서 작성 후 가까운 대리점을 통해 진행 가능합니다.\n\n\n- Q: 코로나 19도 보험 적용이 되나요?\n- A: 코로나 19는 보험 대상에 포함되지 않습니다.\n\n\n- Q: 보험금 지급이 거부되는 경우는 어떤 경우인가요?\n- A: 보험금 지급 거부 사유는 보험 계약서에 명시되어 있습니다. 자세한 사항은 계약서를 참조하세요\n        '),
  Document(id='6d22da05-1870-442c-81aa-9d43d9a45140', metadata={'domain': 'records'}, page_content='# 환자 의료 기록\n- 이름: 홍길동\n- 성별: 남\n- 나이: 30세\n## 진료 내역\n- 2021년 1월 13일: 감기로 인한 발열로 병원 방문\n- 2022년 3월 15일: 비염 진단으로 약 처방\n- 2022년 5월 20일: 피부과 진료 및 약 처방\n- 2022년 6월 10일: 발열 및 기침으로 코로나 검사 및 음성 판정\n- 2022년 7월 2일: 코로나 진단 검사 및 양성 판정\n    ')],
 'answer': '제공된 환자 의료 기록의 진료 내역입니다.\n- 이름: 홍길동, 성별: 남, 나이: 30세\n- 2021-01-13: 감기로 인한 발열로 병원 방문\n- 2022-03-15: 비염 진단, 약 처방\n- 2022-05-20: 피부과 진료, 약 처방\n- 2022-06-10: 발열·기침, 코로나 검사 음성\n- 2022-07-02: